<a href="https://colab.research.google.com/github/lhiwi/AI-Principles_-advanced-searching/blob/main/UCS_fig2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Uniform cost search**

# *A state space graph with backward cost for the traveling Ethiopia search problem*


In [14]:
import heapq

class EthiopiaGraph:
    def __init__(self):
        self.graph = {}

    def add_connection(self, city1, city2, cost):
        # A bi-directional road (edge) between cities with a given cost.
        if city1 not in self.graph:
            self.graph[city1] = []
        if city2 not in self.graph:
            self.graph[city2] = []
        self.graph[city1].append((city2, cost))
        self.graph[city2].append((city1, cost))

    def get_neighbors(self, city):
        return self.graph.get(city, [])


In [10]:
ethiopia_map = EthiopiaGraph()

# connections
ethiopia_map.add_connection("Addis Ababa", "Debre Birhan", 3)
ethiopia_map.add_connection("Addis Ababa", "Ambo", 8)
ethiopia_map.add_connection("Addis Ababa", "Adama", 4)
ethiopia_map.add_connection("Addis Ababa", "Jimma", 9)

ethiopia_map.add_connection("Debre Birhan", "Dessie", 11)
ethiopia_map.add_connection("Debre Birhan", "Debre Sina", 5)
ethiopia_map.add_connection("Dessie", "Lalibela", 17)
ethiopia_map.add_connection("Dessie", "Woldia", 8)
ethiopia_map.add_connection("Lalibela", "Gondar", 15)
ethiopia_map.add_connection("Lalibela", "Sekota", 10)

ethiopia_map.add_connection("Axum", "Gondar", 20)
ethiopia_map.add_connection("Axum", "Shire", 8)
ethiopia_map.add_connection("Gondar", "Bahir Dar", 10)

ethiopia_map.add_connection("Adama", "Asella", 7)
ethiopia_map.add_connection("Adama", "Batu", 6)
ethiopia_map.add_connection("Asella", "Dodola", 9)
ethiopia_map.add_connection("Dodola", "Bale", 13)
ethiopia_map.add_connection("Bale", "Goba", 5)
ethiopia_map.add_connection("Goba", "Sof Oumer", 18)

ethiopia_map.add_connection("Babile", "Harar", 3)
ethiopia_map.add_connection("Babile", "Jijiga", 5)

ethiopia_map.add_connection("Jimma", "Bonga", 7)
ethiopia_map.add_connection("Bonga", "Mizan Teferi", 6)

ethiopia_map.add_connection("Arba Minch", "Wolaita Sodo", 10)
ethiopia_map.add_connection("Wolaita Sodo", "Hossana", 8)


In [ ]:
ethiopia_cities = dict( {'Addis Ababa': {'Adama':3, 'Ambo':5, 'Debre Berhan':5},
             'Adama': {'Matahara':3, 'Asella':4, 'Batu':4, 'Addis Ababa':3},
             'Ambo': {'Wolkite':6, 'Addis Ababa':9, 'Nekemte':5},
             'Debre Berhan': {'Addis Ababa':5, 'Debre Sina':2},
             'Matahara': {'Adama':3, 'Awash':1},
             'Asella': {'Adama':4, 'Assasa':4},
             'Batu': {'Adama':4, 'Buta Jirra':2, 'Shashamene':3},
             'Wolkite': {'Ambo':6, 'Worabe':5, 'Jimma':8},
             'Nekemte': {'Ambo':9, 'Bedelle':5, 'Gimbi':4},
             'Debre Sina': {'Debre Berhan':2, 'Kemise':6, 'Debre Markos':17},
             'Awash': {'Chiro':4, 'Gobi Rasu':5, 'Matahara':1},
             'Assasa': {'Asella':4, 'Dodolla':1},
             'Buta Jirra': {'Batu':2, 'Worabe':2},
             'Shashamene': {'Batu':3, 'Hawassa':1, 'Dodolla':3, 'Hossana':7},
             'Worabe': {'Wolkite':5, 'Hossana':2, 'Buta Jirra':2},
             'Jimma': {'Wolkite':8, 'Bonga':4, 'Bedelle':7},
            'Bedelle': {'Nekemte':5, 'Gore':6, 'Jimma':7},
             'Gimbi': {'Nekemte':4, 'Dambidollo':6},
             'Kemise': {'Debre Sina':6, 'Dessie':4},
             'Debre Markos': {'Debre Sina':17, 'Finote Selam':3},
             'Chiro': {'Awash':4, 'Dire Dawa':8},
             'Gobi Rasu': {'Awash':5, 'Samara':9},
             'Dodolla': {'Assasa':1, 'Shashamene':3, 'Bale':13},
             'Hawassa': {'Shashamene':1, 'Dilla':3},
             'Hossana': {'Shashamene':7, 'Worabe':2, 'Wolaita Sodo':4},
             'Bonga': {'Jimma':4, 'Dawro':10, 'Tepi':8, 'Mizan Teferi':4},
             'Gore': {'Tepi':9, 'Gambella':5, 'Bedelle':6},
             'Dambidollo': {'Gimbi':6, 'Assosa':12, 'Gambella':4},
            'Dessie': {'Kemise':4, 'Woldia':6},
            'Finote Selam': {'Debre Markos':3, 'Bahirdar':6, 'Injibara':2},
             'Dire Dawa': { 'Chiro':8, 'Harar':4},
             'Samara': { 'Gobi Rasu':9, 'Fanti Rasu':7, 'Alamata':11, 'Woldia':8},
            'Bale': {'Liben':11, 'Dodolla':13, 'Goba':18, 'Sof Oumer':23},
             'Dilla': {'Hawassa':3, 'Bulehora':4},
             'Wolaita Sodo': {'Arba Minchi':5, 'Dawro':6, 'Hossana':4},
             'Dawro': { 'Bonga':10, 'Wolaita Sodo':10},
             'Tepi': {'Gore':9, 'Bonga':8, 'Mizan Teferi':4},
            'Mizan Teferi': {'Tepi':4, 'Bonga':4},
             'Gambella': {'Gore':5, 'Dambidollo':4},
             'Assosa': {'Dambidollo':12},
            'Woldia': {'Dessie':6, 'Lalibella':7, 'Samara':8, 'Alamata':3},
             'Bahirdar': {'Finote Selam':6, 'Injibara':4, 'Metekel':11, 'Azezo':7, 'Debre Tabor':4},
             'Injibara': {'Bahirdar':4, 'Finote Selam':2},
             'Harar': { 'Dire Dawa':4, 'Babile':2},
             'Fanti Rasu': {'Samara':7, 'Kilbet Rasu':6},
             'Alamata': {'Samara':11, 'Woldia':3, 'Mekelle':5, 'Sekota':6},
             'Liben': {'Bale':11},
             'Goba': {'Bale':18, 'Sof Oumer':6, 'Babile':28},
             'Sof Oumer': {'Goba':6, 'Bale':23, 'Gode':23},
            'Bulehora': { 'Dilla':4, 'Yabello':3},
            'Arba Minchi': {'Wolaita Sodo':5, 'Konso':4, 'Basketo':10},
             'Basketo': { 'Arba Minchi':10, 'Benchi Maji':5},
             'Metekel': { 'Bahirdar':11},
             'Lalibella': {'Woldia':7, 'Debre Tabor':8, 'Sekota':6},
             'Debre Tabor': {'Lalibella':8, 'Bahirdar':4},
             'Azezo': {'Gondar':1, 'Bahirdar':7, 'Metema':7},
             'Babile': { 'Harar':2, 'Jigjiga':3,'Goba':28},
             'Kilbet Rasu': {'Fanti Rasu':6},
             'Mekelle': {'Alamata':5, 'Adwa':7, 'Adigrat':4, 'Sekota':9},
             'Sekota': {'Alamata':6, 'Mekelle':9, 'Lalibella':6},
            'Dega Habur': {'Jigjiga':5, 'Kebri Dehar':6},
            'Kebri Dehar': {'Gode':5, 'Dega Habur':6, 'Werdez':6},
            'Yabello': { 'Bulehora':3, 'Konso':3, 'Moyale':6},
            'Konso': {'Arba Minchi':4, 'Yabello':3},
            'Benchi Maji': { 'Basketo':5},
            'Gondar': { 'Azezo':1, 'Humera':9, 'Metema':7, 'Debarke':4},
            'Metema': { 'Azezo':7, 'Gondar':7},
            'Jigjiga': { 'Babile':3, 'Dega Habur':5},
            'Adwa': { 'Mekelle':7, 'Axum':1, 'Adigrat':4},
            'Adigrat': { 'Mekelle':4, 'Adwa':4},
            'Gode': { 'Dollo':17, 'Kebri Dehar':5, 'Sof Oumer':23 },
            'Werdez': { 'Kebri Dehar':6},
            'Moyale': { 'Yabello':6},
            'Debarke': { 'Gondar':4, 'Shire':7},
            'Axum': {'Shire':2, 'Adwa':1},
            'Dollo': { 'Gode':17},
            'Shire': { 'Axum':2, 'Humera':8, 'Debarke':7},
            'Humera': { 'Shire':8, 'Gondar':9}})

# **Uniform cost search algorithm implemented**

In [13]:
class UniformCostSearch:
    def __init__(self, graph):
        self.graph = graph

    def find_path(self, start, goal): #UCS from initial to goal state, returning the optimal path and cost.

        priority_queue = [(0, start, [start])]
        visited = set()

        while priority_queue:
            cost, city, path = heapq.heappop(priority_queue)

            if city in visited:
                continue
            visited.add(city)

            # Goal Reached
            if city == goal:
                return cost, path

            # Expanding Neighbors
            for neighbor, edge_cost in self.graph.get_neighbors(city):
                if neighbor not in visited:
                    heapq.heappush(priority_queue, (cost + edge_cost, neighbor, path + [neighbor]))

        return float("inf"), []  # No path found


In [12]:
# Find shortest path
ucs_solver = UniformCostSearch(ethiopia_map)
cost, path = ucs_solver.find_path("Addis Ababa", "Lalibela")

# Print results
print(f"🛣️ Shortest Path to Lalibela: {' -> '.join(path)}")
print(f"🔢 Total Cost: {cost}")


🛣️ Shortest Path to Lalibela: Addis Ababa -> Debre Birhan -> Dessie -> Lalibela
🔢 Total Cost: 34


In [15]:
class MultiGoalUCS:
    def __init__(self, graph):
        """Initialize UCS for multiple goals."""
        self.graph = graph

    def search(self, start, goals):
        """
        Perform UCS to visit all goal states while minimizing cost.
        """
        priority_queue = [(0, start, [start], set())]  # (cost, current_city, path, visited_goals)
        best_solution = None
        best_cost = float("inf")

        while priority_queue:
            cost, city, path, visited_goals = heapq.heappop(priority_queue)

            # If all goal states are visited, update the best solution
            if visited_goals == set(goals):
                if cost < best_cost:
                    best_cost = cost
                    best_solution = path
                continue

            # Expand neighbors
            for neighbor, edge_cost in self.graph.get_neighbors(city):
                new_cost = cost + edge_cost
                new_path = path + [neighbor]
                new_visited_goals = visited_goals.copy()

                # Mark goal states as visited
                if neighbor in goals:
                    new_visited_goals.add(neighbor)

                heapq.heappush(priority_queue, (new_cost, neighbor, new_path, new_visited_goals))

        return best_cost, best_solution


In [16]:
# Initialize UCS and find path
ucs_solver = UniformCostSearch(ethiopia_map)
cost, path = ucs_solver.find_path("Addis Ababa", "Lalibela")

# Print results
print(f"🛣️ Shortest Path to Lalibela: {' -> '.join(path)}")
print(f"🔢 Total Cost: {cost}")


🛣️ Shortest Path to Lalibela: Addis Ababa -> Debre Birhan -> Dessie -> Lalibela
🔢 Total Cost: 34
